In [1]:
import uproot
import pandas as pd
import numpy as np
import boost_histogram as bh
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
folder = '/eos/atlas/atlascerngroupdisk/perf-egamma/InclusivePhotons'
branches = ['evtWeight', 'mcWeight', 'mcTotWeight', 'yWeight', 'y_passOQ', 'y_pt', 'y_eta', 'y_isTruthMatchedPhoton', 'y_convType', 'y_truth_convType', 'y_Rhad1', 'y_Rhad', 'y_Reta', 'y_weta2', 'y_Rphi', 'y_wtots1', 'y_weta1', 'y_fracs1', 'y_deltae', 'y_Eratio', 'y_f1']

In [3]:
def makehadlist(df_name):
    '''Makes the list of values for the combined HadLeakage variable
    where HadLeakage = Rhad if 0.8<|eta|<1.37   and   
                       Rhad1 elsewhere
    df_name is the dataframe
    
    ***could remake this function to just append to the list instead of 
    inserting at specific indices (as long as it goes through everything in order)***'''
    
    hadleaklist = [False] * len(df_name['y_eta'])
    
    for i in range(len(df_name['y_eta'])):
        if abs(df_name['y_eta'][i]) < 0.8 or abs(df_name['y_eta'][i]) > 1.37:
            hadleaklist[i] = df_name['y_Rhad1'][i]
        else:
            hadleaklist[i] = df_name['y_Rhad'][i]
         
    return hadleaklist
        

In [6]:
#Loading in Files
def fileloader(filepath,branches):
    '''for Single Photon root files
    returns a pandas DataFrame'''
    
    file = uproot.open(filepath)
    fileSP = file['SinglePhoton']
    dataframe = fileSP.arrays(branches,library='pd', entry_stop=100)
    
    return dataframe


df_gjnotruth = fileloader(folder+'/mc21_gammajet_v02/PyPt8_inf_mc21_p5058_Rel22_AB22.2.50_v02.root',branches)
# df_jj = fileloader(folder+'/mc21_jetjet_v02/Py8_jetjet_mc21_801279_p5057_Rel22_AB22.2.50_v02.root',branches)

FileNotFoundError: file not found

    '/eos/atlas/atlascerngroupdisk/perf-egamma/InclusivePhotons/mc21_gammajet_v02/PyPt8_inf_mc21_p5058_Rel22_AB22.2.50_v02.root'

Files may be specified as:
   * str/bytes: relative or absolute filesystem path or URL, without any colons
         other than Windows drive letter or URL schema.
         Examples: "rel/file.root", "C:\abs\file.root", "http://where/what.root"
   * str/bytes: same with an object-within-ROOT path, separated by a colon.
         Example: "rel/file.root:tdirectory/ttree"
   * pathlib.Path: always interpreted as a filesystem path or URL only (no
         object-within-ROOT path), regardless of whether there are any colons.
         Examples: Path("rel:/file.root"), Path("/abs/path:stuff.root")

Functions that accept many files (uproot.iterate, etc.) also allow:
   * glob syntax in str/bytes and pathlib.Path.
         Examples: Path("rel/*.root"), "/abs/*.root:tdirectory/ttree"
   * dict: keys are filesystem paths, values are ob

In [ ]:
#adding HadLeakage variable
df_jj['HadLeakage'] = makehadlist(df_jj)
df_gjnotruth['HadLeakage'] = makehadlist(df_gjnotruth)

In [ ]:
#making the good weights:
# df_jj['goodWeight'] = df_jj['mcTotWeight']/df_jj['yWeight']

In [11]:
df_gjnotruth['mcTotWeight'], df_gjnotruth['yWeight'], df_gjnotruth['mcTotWeight']/df_gjnotruth['yWeight'],df_gjnotruth['mcWeight']

(0          0.276039
 1          0.276039
 2          0.276039
 3          0.276039
 4          0.276039
              ...   
 2373008    0.276039
 2373009    0.276039
 2373010    0.276039
 2373011    0.276039
 2373012    0.276039
 Name: mcTotWeight, Length: 2373013, dtype: float64,
 0          0.0
 1          0.0
 2          0.0
 3          0.0
 4          0.0
           ... 
 2373008    0.0
 2373009    0.0
 2373010    0.0
 2373011    0.0
 2373012    0.0
 Name: yWeight, Length: 2373013, dtype: float64,
 0          inf
 1          inf
 2          inf
 3          inf
 4          inf
           ... 
 2373008    inf
 2373009    inf
 2373010    inf
 2373011    inf
 2373012    inf
 Length: 2373013, dtype: float64,
 0          1.0
 1          1.0
 2          1.0
 3          1.0
 4          1.0
           ... 
 2373008    1.0
 2373009    1.0
 2373010    1.0
 2373011    1.0
 2373012    1.0
 Name: mcWeight, Length: 2373013, dtype: float64)

In [7]:
## ONLY SELECTING TRUTH MATCHED photons (g)

df_gj = df_gjnotruth[df_gjnotruth.y_isTruthMatchedPhoton]
df_gj.index = list(range(len(df_gj)))   #resetting indices

In [8]:
#pickling
def picklewrite(file,filename,filepath='picklefiles/'):
    '''writes 'file' to a pickle file with name 'filename' (str)
    automatically into picklefiles folder (filepath)
    
    if want file in present directory, set filepath='' '''
    pickle.dump(file,open(filepath+filename, 'wb'))
    return

In [9]:
#pickling
picklewrite(df_jj,'df_jj_801279.pickle')
picklewrite(df_gjnotruth,'df_gj_800664.pickle')
picklewrite(df_gj,'df_gj_800664_truth.pickle')